# ABOUT MISSING VALUES

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv("melb_data.csv")

In [3]:
data.shape

(13580, 21)

In [4]:
data.head(3)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [5]:
print('persentase nilai null: "{}" \n'.format(data.isnull().sum()/data.shape[0]))

persentase nilai null: "Suburb           0.000000
Address          0.000000
Rooms            0.000000
Type             0.000000
Price            0.000000
Method           0.000000
SellerG          0.000000
Date             0.000000
Distance         0.000000
Postcode         0.000000
Bedroom2         0.000000
Bathroom         0.000000
Car              0.004566
Landsize         0.000000
BuildingArea     0.474963
YearBuilt        0.395803
CouncilArea      0.100810
Lattitude        0.000000
Longtitude       0.000000
Regionname       0.000000
Propertycount    0.000000
dtype: float64" 



In [6]:
y = data.Price
melb_predictors = data.drop(['Price'], axis = 1)
X = melb_predictors.select_dtypes(exclude = ['object'])

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [8]:
missing_val_count_by_columns = (X_train.isnull().sum())
print(missing_val_count_by_columns[missing_val_count_by_columns>0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


In [9]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 10, random_state = 0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## dropping columns 

In [10]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

In [11]:
reduced_X_train = X_train.drop(cols_with_missing, axis = 1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis = 1)

In [12]:
print("MAE from approach 1 (drop columns with missing values): ")
print(score_dataset(reduced_X_train, reduced_X_valid,  y_train, y_valid))

MAE from approach 1 (drop columns with missing values): 
183550.22137772635


## imputation 

In [13]:
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

In [14]:
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [15]:
print("MAE from approach 2 (imputation): ")
print(score_dataset(imputed_X_train, imputed_X_valid,  y_train, y_valid))

MAE from approach 2 (imputation): 
178166.46269899711


## extension to imputation 

In [16]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

In [17]:
for col in cols_with_missing:
    X_train_plus[col + '_was missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was missing'] = X_valid_plus[col].isnull()
    
imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(imputer.transform(X_valid_plus))
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

In [18]:
imputed_X_valid_plus

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was missing,BuildingArea_was missing,YearBuilt_was missing
0,4.0,8.0,3016.0,4.0,2.0,2.0,450.0,190.000000,1910.000000,-37.86100,144.89850,6380.0,0.0,0.0,0.0
1,2.0,6.6,3011.0,2.0,1.0,0.0,172.0,81.000000,1900.000000,-37.81000,144.88960,2417.0,0.0,0.0,0.0
2,3.0,10.5,3020.0,3.0,1.0,1.0,581.0,153.764119,1964.839866,-37.76740,144.82421,4217.0,0.0,1.0,1.0
3,3.0,4.5,3181.0,2.0,2.0,1.0,128.0,134.000000,2000.000000,-37.85260,145.00710,7717.0,0.0,0.0,0.0
4,3.0,8.5,3044.0,3.0,2.0,2.0,480.0,153.764119,1964.839866,-37.72523,144.94567,7485.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2.0,6.4,3011.0,2.0,1.0,1.0,47.0,35.000000,2013.000000,-37.80140,144.89590,7570.0,0.0,0.0,0.0
2712,4.0,8.0,3016.0,4.0,2.0,4.0,551.0,153.764119,1964.839866,-37.85790,144.87860,6380.0,0.0,1.0,1.0
2713,3.0,10.8,3105.0,3.0,1.0,1.0,757.0,153.764119,1964.839866,-37.78094,145.10131,4480.0,0.0,1.0,1.0
2714,4.0,6.2,3039.0,4.0,1.0,3.0,478.0,152.000000,1925.000000,-37.76421,144.90571,6232.0,0.0,0.0,0.0


In [19]:
print("MAE from approach 3 (extention to imputation): ")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus,  y_train, y_valid))

MAE from approach 3 (extention to imputation): 
178927.503183954


## for submission 

In [ ]:
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [ ]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

In [20]:
# #untuk data uji
# # Preprocess test data
# final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# # Get test predictions
# preds_test = model.predict(final_X_test)

In [21]:
# # Save test predictions to file
# output = pd.DataFrame({'Id': X_test.index,
#                        'SalePrice': preds_test})
# output.to_csv('submission.csv', index=False)

# ABOUT CATEGORICAL VARIABLES

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

C:\Users\DELL-POJ\anaconda3\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## dropping categorical variables 

In [25]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


## ordinal encoding 

In [26]:
from sklearn.preprocessing import OrdinalEncoder

In [27]:
# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
165936.40548390493


## one-hot encoding 

In [28]:
from sklearn.preprocessing import OneHotEncoder

In [29]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols])) #hanya kolom bertipe objek/non-numerik
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

#FINAL DATA
# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
166089.4893009678


## for submission 

In [32]:
#HATI HATI TERHADAP DATA TRAIN DAN VALIDASI TERHADAP KATEGORI VARIABEL YANG SEBAGIAN TIDAK TERMUAT
# # All categorical columns
# object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# # Columns that can be safely ordinal encoded
# good_label_cols = [col for col in object_cols if 
#                    set(X_valid[col]).issubset(set(X_train[col]))]
        
# # Problematic columns that will be dropped from the dataset
# bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
# print('Categorical columns that will be ordinal encoded:', good_label_cols)
# print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

In [31]:
# from sklearn.preprocessing import OrdinalEncoder

# # Drop categorical columns that will not be encoded
# label_X_train = X_train.drop(bad_label_cols, axis=1)
# label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# # Apply ordinal encoder 
# ordinal_encoder = OrdinalEncoder()
# label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
# label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])  

In [33]:
# # Get number of unique entries in each column with categorical data
# object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
# d = dict(zip(object_cols, object_nunique))

# # Print number of unique entries by column, in ascending order
# sorted(d.items(), key=lambda x: x[1])